# 🔄 Basic Agent Workflows with GitHub Models (.NET)

## 📋 Workflow Orchestration Tutorial

This notebook demonstrates how to build sophisticated **agent workflows** using the Microsoft Agent Framework for .NET and GitHub Models. You'll learn to create multi-step business processes where AI agents collaborate to accomplish complex tasks through structured orchestration patterns.

## 🎯 Learning Objectives

### 🏗️ **Workflow Architecture Fundamentals**
- **Workflow Builder**: Design and orchestrate complex multi-step AI processes
- **Agent Coordination**: Coordinate multiple specialized agents within workflows
- **GitHub Models Integration**: Leverage GitHub's AI model inference service in workflows
- **Visual Workflow Design**: Create and visualize workflow structures for better understanding

### 🔄 **Process Orchestration Patterns**
- **Sequential Processing**: Chain multiple agent tasks in logical order
- **State Management**: Maintain context and data flow across workflow stages
- **Error Handling**: Implement robust error recovery and workflow resilience
- **Performance Optimization**: Design efficient workflows for enterprise-scale operations

### 🏢 **Enterprise Workflow Applications**
- **Business Process Automation**: Automate complex organizational workflows
- **Content Production Pipeline**: Editorial workflows with review and approval stages
- **Customer Service Automation**: Multi-step customer inquiry resolution
- **Data Processing Workflows**: ETL workflows with AI-powered transformation

## ⚙️ Prerequisites & Setup

### 📦 **Required NuGet Packages**

This workflow demonstration uses several key .NET packages:

```xml
<!-- Core AI Framework -->
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />

<!-- Agent Framework (Local Development) -->
<!-- Microsoft.Agents.AI.dll - Core agent abstractions -->
<!-- Microsoft.Agents.AI.OpenAI.dll - OpenAI/GitHub Models integration -->

<!-- Configuration and Environment -->
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

### 🔑 **GitHub Models Configuration**

**Environment Setup (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

**GitHub Models Access:**
1. Sign up for GitHub Models (currently in preview)
2. Generate a personal access token with model access permissions
3. Configure environment variables as shown above

### 🏗️ **Workflow Architecture Overview**

```mermaid
graph TD
    A[Workflow Builder] --> B[Agent Registry]
    B --> C[Workflow Execution Engine]
    C --> D[Agent 1: Content Generator]
    C --> E[Agent 2: Content Reviewer] 
    D --> F[Workflow Results]
    E --> F
    G[GitHub Models API] --> D
    G --> E
```

**Key Components:**
- **WorkflowBuilder**: Main orchestration engine for designing workflows
- **AIAgent**: Individual specialized agents with specific capabilities
- **GitHub Models Client**: AI model inference service integration
- **Execution Context**: Manages state and data flow between workflow stages

## 🎨 **Enterprise Workflow Design Patterns**

### 📝 **Content Production Workflow**
```
User Request → Content Generation → Quality Review → Final Output
```

### 🔍 **Document Processing Pipeline**
```
Document Input → Analysis → Extraction → Validation → Structured Output
```

### 💼 **Business Intelligence Workflow**
```
Data Collection → Processing → Analysis → Report Generation → Distribution
```

### 🤝 **Customer Service Automation**
```
Customer Inquiry → Classification → Processing → Response Generation → Follow-up
```

## 🏢 **Enterprise Benefits**

### 🎯 **Reliability & Scalability**
- **Deterministic Execution**: Consistent, repeatable workflow outcomes
- **Error Recovery**: Graceful handling of failures at any workflow stage
- **Performance Monitoring**: Track execution metrics and optimization opportunities
- **Resource Management**: Efficient allocation and utilization of AI model resources

### 🔒 **Security & Compliance**
- **Secure Authentication**: GitHub token-based authentication for API access
- **Audit Trails**: Complete logging of workflow execution and decision points
- **Access Control**: Granular permissions for workflow execution and monitoring
- **Data Privacy**: Secure handling of sensitive information throughout workflows

### 📊 **Observability & Management**
- **Visual Workflow Design**: Clear representation of process flows and dependencies
- **Execution Monitoring**: Real-time tracking of workflow progress and performance
- **Error Reporting**: Detailed error analysis and debugging capabilities
- **Performance Analytics**: Metrics for optimization and capacity planning

Let's build your first enterprise-ready AI workflow! 🚀

In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [2]:
#r "nuget: System.ClientModel, 1.6.1.0"

Installed Packages System.ClientModel, 1.6.1

In [3]:
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: OpenTelemetry.Api, 1.0.0"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.Identity, 1.15.0 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [5]:

#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3

In [ ]:

#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [9]:
using System;
using System.ComponentModel;
using System.ClientModel;
using OpenAI;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;

In [10]:
 using DotNetEnv;

In [11]:
Env.Load("../../../.env");

In [12]:

var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri(github_endpoint)
};

In [14]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [15]:
const string ReviewerAgentName = "Concierge";
const string ReviewerAgentInstructions = @"
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. ";

In [16]:
const string FrontDeskAgentName = "FrontDesk";
const string FrontDeskAgentInstructions = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [17]:
AIAgent reviewerAgent = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:ReviewerAgentName,instructions:ReviewerAgentInstructions);
AIAgent frontDeskAgent  = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:FrontDeskAgentName,instructions:FrontDeskAgentInstructions);

In [18]:
var workflow = new WorkflowBuilder(frontDeskAgent)
            .AddEdge(frontDeskAgent, reviewerAgent)
            .Build();

In [19]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent("I would like to go to Paris.") 
]);

In [20]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, userMessage);

In [21]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
string id="";
string messageData="";
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
    if (evt is AgentRunUpdateEvent executorComplete)
    {
        if(id=="")
        {
            id=executorComplete.ExecutorId;
        }
        if(id==executorComplete.ExecutorId)
        {
            messageData+=executorComplete.Data.ToString();
        }
        else
        {
            id=executorComplete.ExecutorId;
        }
        // Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
    }
}

Console.WriteLine(messageData);

Visit the Louvre Museum. It's a must-see for art enthusiasts and history lovers.That recommendation is quite popular and likely to attract many tourists. To refine it for a more local and authentic experience, consider suggesting an alternative that focuses on smaller, lesser-known art venues or galleries. Look for places where local artists exhibit or community spaces that host cultural events. This approach allows travelers to connect with the local art scene more intimately, away from the typical tourist routes.
